In [199]:
#Importación de Librerías
import pandas as pd
from string import ascii_uppercase as alfabeto
import pickle

In [200]:
#Guardar las tablas de Grupos del mundial 2022
tablas = pd.read_html('https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

In [201]:
tablas[12]

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Qatar (H),0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Ecuador,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Senegal,0,0,0,0,0,0,0,0,NaN
3,4,Netherlands,0,0,0,0,0,0,0,0,NaN


In [202]:
#Generamos un diccionario para guardar las tablas con la letra correspondiente
diccionario_tablas = {}
for letra, i in zip(alfabeto, range(12, 68, 7)):
  df = tablas[i]
  df.rename(columns={'Teamvte':'Team'},inplace=True)
  df.pop('Qualification')
  diccionario_tablas[f'Group {letra}'] = df


In [203]:
diccionario_tablas['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Qatar (H),0,0,0,0,0,0,0,0
1,2,Ecuador,0,0,0,0,0,0,0,0
2,3,Senegal,0,0,0,0,0,0,0,0
3,4,Netherlands,0,0,0,0,0,0,0,0


In [204]:
#Vaciamos nuestro diccionario en un archivo llamado dict_table
with open('dict_table','wb') as output:
  pickle.dump(diccionario_tablas, output)

In [205]:
#Importamos bs4 para scrapping
from bs4 import BeautifulSoup
import requests

In [206]:
#Definimos los años de los mundiales a buscar
years = [1930, 1934, 1938, 1942, 1946, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018]

In [207]:
#Función para obtener los resultados de los partidos
def obtener_partidos(year):
  if year == '2022':
    web = 'https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup#Group_stage'
  else:
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'

  response = requests.get(web)
  content = response.text
  soup = BeautifulSoup(content, 'lxml')

  matches = soup.findAll('div', class_='footballbox')

  local = []
  score = []
  visit = []

  for match in matches:
    local.append(match.find('th', class_='fhome').get_text())
    score.append(match.find('th', class_='fscore').get_text())
    visit.append(match.find('th', class_='faway').get_text())

  diccionario_football = {'local': local, 'score': score, 'visit': visit}

  df_football = pd.DataFrame(diccionario_football)
  df_football['year'] = year
  return df_football

In [208]:
#Guardamos los datos históricos
fifa = [obtener_partidos(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv('fifa_worldcup_historical_data.csv', index=False, encoding='utf-8-sig')

In [209]:
#Guardamos los partidos a jugarse en 2022
df_fixture = obtener_partidos('2022')
df_fixture.to_csv('fifa_worldcup_fixture.csv', index=False, encoding='utf-8-sig')

In [210]:
#Limpieza de datos
df_historico = pd.read_csv('fifa_worldcup_historical_data.csv')
df_fixture = pd.read_csv('fifa_worldcup_fixture.csv')

In [211]:
#Limpiando fixture
df_fixture['local'] = df_fixture['local'].str.strip()
df_fixture['visit'] = df_fixture['visit'].str.strip()

In [212]:
index_eliminar = df_historico[df_historico['local'].str.contains('Sweden') &
             df_historico['visit'].str.contains('Austria')].index

In [213]:
df_historico.drop(index=index_eliminar, inplace=True)

In [214]:
#Limpiando fixture
df_historico['local'] = df_historico['local'].str.strip()
df_historico['visit'] = df_historico['visit'].str.strip()

In [215]:
#df_historico[df_historico['score'].str.contains('[^\d–]')]
df_historico['score'] = df_historico['score'].str.replace('[^\d–]', '', regex=True)

In [216]:
df_historico

,local,score,visit,year
0,France,4–1,Mexico,1930
1,Argentina,1–0,France,1930
2,Chile,3–0,Mexico,1930
3,Chile,1–0,France,1930
4,Argentina,6–3,Mexico,1930
...,...,...,...,...
860,Russia,2–2,Croatia,2018
861,France,1–0,Belgium,2018
862,Croatia,2–1,England,2018
863,Belgium,2–0,England,2018


In [217]:
df_historico[['localGol','visitGol']] = df_historico['score'].str.split('–', expand=True)

In [218]:
df_historico.drop('score', axis=1, inplace=True)

In [219]:
df_historico

,local,visit,year,localGol,visitGol
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3
...,...,...,...,...,...
860,Russia,Croatia,2018,2,2
861,France,Belgium,2018,1,0
862,Croatia,England,2018,2,1
863,Belgium,England,2018,2,0


In [220]:
#df_historico.dtypes
df_historico = df_historico.astype({'localGol' : int, 'visitGol': int, 'year': int})

In [221]:
df_historico['golesTotales'] = df_historico['localGol'] + df_historico['visitGol']

In [222]:
df_historico

,local,visit,year,localGol,visitGol,golesTotales
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
860,Russia,Croatia,2018,2,2,4
861,France,Belgium,2018,1,0,1
862,Croatia,England,2018,2,1,3
863,Belgium,England,2018,2,0,2


In [223]:
df_historico.to_csv('clean_fifa_worldcup_historical_data.csv', index=False)
df_fixture.to_csv('clean_fifa_worldcup_fixture.csv', index=False)

In [224]:
from scipy.stats import poisson

In [225]:
dict_table = pickle.load(open('dict_table', 'rb'))
df_historico = pd.read_csv('clean_fifa_worldcup_historical_data.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

In [226]:
df_historico

,local,visit,year,localGol,visitGol,golesTotales
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
859,Russia,Croatia,2018,2,2,4
860,France,Belgium,2018,1,0,1
861,Croatia,England,2018,2,1,3
862,Belgium,England,2018,2,0,2


In [227]:
df_local = df_historico[['local','localGol','visitGol']]
df_visit = df_historico[['visit','localGol','visitGol']]

In [228]:
df_local = df_local.rename(columns={'local':'Equipo', 'localGol':'golesAnotados', 'visitGol':'golesRecibidos'})
df_visit = df_visit.rename(columns={'visit':'Equipo', 'localGol':'golesRecibidos', 'visitGol':'golesAnotados'})

In [229]:
df_team_stats = pd.concat([df_local, df_visit], ignore_index=True).groupby('Equipo').mean()
df_team_stats = df_team_stats.replace('Qatar', 'Qatar (H)')
df_team_stats

,golesAnotados,golesRecibidos
Equipo,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.717949,1.166667
Australia,0.812500,1.937500
Austria,1.576923,1.692308
...,...,...
Uruguay,1.603774,1.339623
Wales,0.800000,0.800000
West Germany,2.050847,1.254237


In [230]:
def prediccion_puntos(local, visit):
  if local in df_team_stats.index and visit in df_team_stats.index:
    #Goles anotados * goles recibidos
    lamb_local = df_team_stats.at[local, 'golesAnotados'] * df_team_stats.at[visit, 'golesRecibidos']
    lamb_visit = df_team_stats.at[visit, 'golesAnotados'] * df_team_stats.at[local, 'golesRecibidos']
    prob_local, prob_visit, prob_draw = 0,0,0
    for x in range(0,11): #Goles local
      for y in range(0,11): #Goles visitante
        p = poisson.pmf(x, lamb_local) * poisson.pmf(y, lamb_visit)
        if x == y:
          prob_draw += p
        elif x > y:
          prob_local += p
        else:
          prob_visit += p

    puntos_local = 3 * prob_local + prob_draw
    puntos_visit = 3 * prob_visit + prob_draw
    return (puntos_local, puntos_visit)
  else:
    return(0,0)

In [231]:
prediccion_puntos('Qatar (H)','France')

(0, 0)

In [232]:
df_fixture = df_fixture.replace('Qatar', 'Qatar (H)')
df_fixture['local'] = df_fixture['local'].str.strip()
df_fixture['visit'] = df_fixture['visit'].str.strip()

In [233]:
#Dividir fixture en grupos
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_oct = df_fixture[48:56].copy()
df_fixture_quarters = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [234]:
#Correr partidos de fase de grupos y actualizar tablas
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['local'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        local, visit = row['local'], row['visit']
        points_local, points_visit = prediccion_puntos(local, visit)
        dict_table[group].loc[dict_table[group]['Team'] == local, 'Pts'] += points_local
        dict_table[group].loc[dict_table[group]['Team'] == visit, 'Pts'] += points_visit
        dict_table[group]['Pts'] = dict_table[group]['Pts'].astype(float)

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

<ipython-input-234-3de7b82310a4>:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.39716107]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == local, 'Pts'] += points_local
<ipython-input-234-3de7b82310a4>:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.78567341]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == local, 'Pts'] += points_local
<ipython-input-234-3de7b82310a4>:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.1926621]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[gr

In [235]:
dict_table

{'Group A':           Team  Pts
 0  Netherlands  4.0
 1      Senegal  2.0
 2      Ecuador  2.0
 3    Qatar (H)  0.0,
 'Group B':             Team  Pts
 0        England  6.0
 1          Wales  5.0
 2  United States  4.0
 3           Iran  2.0,
 'Group C':            Team  Pts
 0     Argentina  7.0
 1        Poland  6.0
 2        Mexico  4.0
 3  Saudi Arabia  1.0,
 'Group D':         Team  Pts
 0     France  7.0
 1    Denmark  6.0
 2    Tunisia  3.0
 3  Australia  2.0,
 'Group E':          Team  Pts
 0     Germany  7.0
 1       Spain  5.0
 2       Japan  3.0
 3  Costa Rica  2.0,
 'Group F':       Team  Pts
 0  Croatia  7.0
 1  Belgium  5.0
 2  Morocco  4.0
 3   Canada  0.0,
 'Group G':           Team  Pts
 0       Brazil  8.0
 1  Switzerland  4.0
 2       Serbia  3.0
 3     Cameroon  2.0,
 'Group H':           Team  Pts
 0     Portugal  6.0
 1      Uruguay  5.0
 2        Ghana  4.0
 3  South Korea  2.0}

In [236]:
dict_table['Group A']

,Team,Pts
0,Netherlands,4.0
1,Senegal,2.0
2,Ecuador,2.0
3,Qatar (H),0.0


In [237]:
#Generar octavos
df_fixture_oct

,local,score,visit,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [238]:
for group in dict_table:
  group_winner = dict_table[group].loc[0, 'Team']
  runners_up = dict_table[group].loc[1, 'Team']
  df_fixture_oct.replace({f'Winners {group}': group_winner,
                          f'Runners-up {group}': runners_up}, inplace=True)

df_fixture_oct['winner'] = '¿?'

In [239]:
df_fixture_oct

,local,score,visit,year,winner
48,Netherlands,Match 49,Wales,2022,¿?
49,Argentina,Match 50,Denmark,2022,¿?
50,France,Match 52,Poland,2022,¿?
51,England,Match 51,Senegal,2022,¿?
52,Germany,Match 53,Belgium,2022,¿?
53,Brazil,Match 54,Uruguay,2022,¿?
54,Croatia,Match 55,Spain,2022,¿?
55,Portugal,Match 56,Switzerland,2022,¿?


In [240]:
#Obtener ganador
def obtener_ganador(df_fixture_updated):
  for index, row in df_fixture_updated.iterrows():
    local, visit = row['local'], row['visit']
    points_local, points_visit = prediccion_puntos(local, visit)
    if points_local > points_visit:
      winner = local
    else:
      winner = visit
    df_fixture_updated.loc[index, 'winner'] = winner
  return df_fixture_updated

In [241]:
obtener_ganador(df_fixture_oct)

,local,score,visit,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Croatia
55,Portugal,Match 56,Switzerland,2022,Portugal


In [242]:
#Cuartos de final
df_fixture_quarters

,local,score,visit,year
56,Winners Match 53,Match 58,Winners Match 54,2022
57,Winners Match 49,Match 57,Winners Match 50,2022
58,Winners Match 55,Match 60,Winners Match 56,2022
59,Winners Match 51,Match 59,Winners Match 52,2022


In [243]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [244]:
update_table(df_fixture_oct, df_fixture_quarters)

,local,score,visit,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Croatia,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [245]:
obtener_ganador(df_fixture_quarters)

,local,score,visit,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Croatia,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


In [246]:
#Semifinal
update_table(df_fixture_quarters, df_fixture_semi)

,local,score,visit,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [247]:
obtener_ganador(df_fixture_semi)

,local,score,visit,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


In [248]:
update_table(df_fixture_semi, df_fixture_final)

,local,score,visit,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [249]:
obtener_ganador(df_fixture_final)


,local,score,visit,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
